# Aggregate and Cloudmask batch workflow
## Applied to DinoV3 CHMs
Initially, aggregate only (no cloudmask)

Matt Macander

2026-01-08

cloudmask for these chm
/explore/nobackup/projects/above/misc/ABoVE_Shrubs/cloudmask/002m_2023

can also do other products (or already have done other products?)

Later, update using newer cloud mask.

W

# Setup

In [1]:
library(sf)
library(tidyverse)
library(janitor)
library(magrittr)
library(lubridate)
library(fs)
library(glue)
library(readxl)
library(terra)
library(furrr)
library(DT)
library(tictoc)
library(gdalUtilities)
# library(mapview)
# library(spatialEco)
# library(gfcanalysis)
srliteBase <- path('/explore/nobackup/projects/above/misc/ABoVE_Shrubs')

Linking to GEOS 3.11.1, GDAL 3.6.1, PROJ 9.1.0; sf_use_s2() is TRUE

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   1.0.0 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.5.0 
✔ readr   2.1.3      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test



Attaching package: ‘magrittr’


The following object is masked from ‘package:purrr’:

    set_names


The following object is masked from ‘package:tidyr’:

    extract


Loading required package: timechange


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


terra 1.5.21


Attaching package: ‘terra’


The following object is masked from ‘packa

In [2]:
#GDAL path fixup on ADAPT, may just be me
secondPath <- sf_proj_search_paths()[[2]]
secondPath
sf_proj_search_paths(secondPath)

[1] "/app/jupyter/lab/x86/dev/share/proj"

[1] "/app/jupyter/lab/x86/dev/share/proj"

In [3]:
# Define template ('snap') rasters for different resolutions
# Corners don't matter except for the pixel alignmnet they imply
# Set CRS to a regional CRS so products from multiple UTM zones can be mosaicked
template100m_3338 <- rast(xmin=-1500,ymin=-1500,xmax=1500,ymax=1500, resolution=100, crs='EPSG:3338', vals=1)
template010m_3338 <- rast(xmin=-1005,ymin=-1005,xmax=1005,ymax=1005, resolution=10, crs='EPSG:3338', vals=1)

# Generate table of paths and filenames for various products, and SRLite Metrics
## Cloudmask List (not run initially in 2026)
/explore/nobackup/projects/above/misc/ABoVE_Shrubs/cloudmask/omni_cloudmask/002m_2023/

In [4]:
# cm_fs <- tibble(cmPath = dir_ls('/explore/nobackup/projects/above/misc/ABoVE_Shrubs/cloudmask/002m', recurse=T, type='file', glob='*.tif')) %>%
cm_fs <- tibble(cmPath = dir_ls('/explore/nobackup/projects/above/misc/ABoVE_Shrubs/cloudmask/omni_cloudmask/002m_2023/', recurse=T, type='file', glob='*.tif')) %>%
mutate(cmFile = path_file(cmPath),
      catid = str_sub(cmFile, 20, 35),
      suffix = str_sub(cmFile, 36, -1)) %>%
arrange(cmFile) %>%
filter(suffix != '-toa.cloudmask_ndv.tif')

cm_fs %>% group_by(suffix) %>% tally()
cm_fs %>% group_by(catid) %>% tally() %>% group_by(n) %>% tally()


suffix,n
<chr>,<int>
-sr-02m_OCM_v1_6_0.tif,3219


Storing counts in `nn`, as `n` already present in input
ℹ Use `name = "new_name"` to pick a new name.


n,nn
<int>,<int>
1,3219


# Canopy Height Model, v2026 DinoV3
our final CHMs are here: 

/explore/nobackup/projects/above/misc/ABoVE_Shrubs/development/chm/dinov3/4.3.2.5

here is our validation dir: 

/explore/nobackup/projects/above/misc/ABoVE_Shrubs/validation/chm/dinov3/4.3.2.5


In [5]:
chm_fs <- tibble(chmPath = dir_ls('/explore/nobackup/projects/above/misc/ABoVE_Shrubs/development/chm/dinov3/4.3.2.5', recurse=T, type='file', glob='*.tif')) %>%
mutate(chmFile = path_file(chmPath),
      catid = str_sub(chmFile, 20, 35),
      suffix = str_sub(chmFile, 36, -1)
      ) %>%
arrange(chmFile) %>%
    mutate(
           # chmFile = path_file(chmPath),
           # catid = str_sub(chmFile,20,35),
           # nameBase = str_sub(path_ext_remove(path_file(chmPath)), 0, -19),
           nameBase = str_sub(path_ext_remove(path_file(chmPath)), 0, -12),
           # chmOut_100m = path(srliteBase, 'chm/2026_chm/4.3.2.5/agg100m', glue('{nameBase}-chm-100m.tif')),
           chmDmOutTmp = path(srliteBase, 'chm/2026_chm/4.3.2.5/dm_10m', glue('{nameBase}-chm-dm-10m-tmp.tif')),
           chmDmOut = path(srliteBase, 'chm/2026_chm/4.3.2.5/dm_10m', glue('{nameBase}-chm-dm-10m.tif'))
)
# filter(suffix != '-toa.cloudmask_ndv.tif')

chm_fs %>% group_by(suffix) %>% tally()
chm_fs %>% group_by(catid) %>% tally() %>% group_by(n) %>% tally()

# chm_fs

datatable(head(chm_fs))

suffix,n
<chr>,<int>
-sr-02m.chm.tif,3219


Storing counts in `nn`, as `n` already present in input
ℹ Use `name = "new_name"` to pick a new name.


n,nn
<int>,<int>
1,3219


HTML widgets cannot be represented in plain text (need html)

In [6]:
# chms_predicted_2026_fs <- bind_rows(
#     tibble(chmPath = dir_ls('/explore/nobackup/projects/above/misc/ABoVE_Shrubs/development/chm/dinov3/4.3.2.5', glob='*/*.tif', recurse=T)) %>%
#         mutate(chmSource = 'CHM DinoV3 Model v2026 4.3.2.5, Inference Strips',
#                chmSourceShort = 'inference_strips'),
#     # tibble(chmPath = dir_ls('/explore/nobackup/projects/above/misc/ABoVE_Shrubs/validation/chm/dinov3/4.3.2.5', glob='*/*.tif', recurse=T)) %>%
#     #     mutate(chmSource = 'CHM DinoV3 Model v2026 4.3.2.5, Validation Strips',
#     #            chmSourceShort = 'validation_strips')) %>%
#     mutate(chmFile = path_file(chmPath),
#            catid = str_sub(chmFile,20,35),
#            # nameBase = str_sub(path_ext_remove(path_file(chmPath)), 0, -19),
#            nameBase = str_sub(path_ext_remove(path_file(chmPath)), 0, -12),
#            # chmOut_100m = path(srliteBase, 'chm/2026_chm/4.3.2.5/agg100m', glue('{nameBase}-chm-100m.tif')),
#            chmDmOutTmp = path(srliteBase, 'chm/2026_chm/4.3.2.5/dm_10m', glue('{nameBase}-chm-dm-10m-tmp.tif')),
#            chmDmOut = path(srliteBase, 'chm/2026_chm/4.3.2.5/dm_10m', glue('{nameBase}-chm-dm-10m.tif'))
# )

# chms_predicted_2026_fs %>% group_by(chmSource) %>% tally()

# datatable(head(chms_predicted_2026_fs))

In [7]:
# unique_joined <- cm_fs %>% select(catid, stripName, cmPath, cmFile) %>%
unique_joined <- cm_fs %>% select(catid, cmPath, cmFile) %>%
left_join(chm_fs %>% select(catid, chmPath, chmFile, chmDmOutTmp, chmDmOut)) #%>%
# left_join(toa_fp_wMeta) %>%
# left_join(cm_fs %>% select(catid, cmPath, cmFile)) %>%
# left_join(ccdc_fs %>% select(catid, ccdcPath, ccdcFile)) %>%
# left_join(chms_predicted_fs %>% select(catid, chmPath, chmFile, chmSource))
# st_as_sf()

colnames(unique_joined)

print(c('SRLite rows:', nrow(unique_joined)))
# nrow(srlite_unique_joined %>% filter(!is.na(toaFile) & !is.na(cmFile)))

# unmatched <- unique_joined %>%
# anti_join(unique_joined %>% filter(!is.na(toaFile) & !is.na(cmFile)) %>% select(catid)) #%>% st_drop_geometry()
# print(c('SRLite rows without cloudmask and/or TOA:', nrow(unmatched)))
# datatable(unmatched)

# srlite_unique_joined_matched <- srlite_unique_joined %>%
# anti_join(unmatched, by='catid')

print(c('cm rows with chm:', nrow(unique_joined)))
datatable(head(unique_joined), caption='unique_joined chm and cm')

# print(c('SRLite rows with cloudmask and TOA:', nrow(srlite_unique_joined_matched)))
# datatable(head(srlite_unique_joined_matched), caption='SRLite and matching TOA/CM')

# write_csv(srlite_unique_joined_matched, # %>% select(-starts_with('chm')),# %>% st_drop_geometry(),
# '/explore/nobackup/projects/above/misc/ABoVE_Shrubs/footprints/srlite_unique_x_toa_cm_ccdc_chm_v20231031.csv')
# write_sf(srlite_unique_joined %>% select(-starts_with('chm')), '/explore/nobackup/projects/above/misc/ABoVE_Shrubs/footprints/srlite_unique_x_toa_cm_ccdc_v20231030.gpkg')

Joining, by = "catid"


[1] "catid"       "cmPath"      "cmFile"      "chmPath"     "chmFile"    
[6] "chmDmOutTmp" "chmDmOut"

[1] "SRLite rows:" "3219"        
[1] "cm rows with chm:" "3219"             


HTML widgets cannot be represented in plain text (need html)

# Aggregate CHM strips to 10m resolution, convert to decimeter

In [8]:
# 2026-01-27 Modified to account for latest CHM inputs being pre-scaled to dm

# agg_and_cloudmask <- function(toaPath, srlitePath, cmPath, templateRaster, resName, rastid) {
agg_and_cloudmask <- function(chmPath, cmPath, chmOut, chmOutTmp, templateRaster) { #, resName) {
    nameBase = str_sub(path_ext_remove(path_file(chmPath)), 0, 35)
    # chmOut = path(srliteBase, glue('toa/agg{resName}_cm'), glue('{nameBase}-toa-cm-{resName}.tif'))
    # chmOutTmp = glue('{path_ext_remove(chmOut)}-tmp.tif')
    # toaOut = path(srliteBase, glue('toa/agg{resName}_cm'), glue('{nameBase}-toa-cm-{resName}.tif'))
    # toaOutTmp = glue('{path_ext_remove(toaOut)}-tmp.tif')
    # srliteOut = path(srliteBase, glue('srlite/agg{resName}_cm'), glue('{nameBase}-sr-cm-{resName}.tif'))
    # srliteOutTmp = glue('{path_ext_remove(srliteOut)}-tmp.tif')
    # rastidOut = path(srliteBase, glue('rastid/agg{resName}_cm'), glue('{nameBase}-rastid-cm-{resName}.tif'))
    # rastidOutTmp = glue('{path_ext_remove(rastidOut)}-tmp.tif')
    # print(c(toaPath, nameBase, toaOut))
    if(!file_exists(chmOut)) {
    # if(!(file_exists(toaOut) & file_exists(srliteOut))) {
        cm <- rast(cmPath)
        chm <- rast(chmPath)
        # toa <- rast(toaPath)
        # srlite <- rast(srlitePath)
        # Convert 8band inputs to 4band, this assumes band-ordering is stable
        # if (nlyr(toa) == 8) { toa <- subset(toa, c(2,3,5,7)) }
        # if (nlyr(srlite) == 8) { srlite <- subset(srlite, c(1,2,3,4)) }

        # Reproject cloudmask to template raster resolution
        # Cloud = 1, Cloudfree = 0 so average raster = cloud proportion 0-1
        pCloudAgg <- project(cm, templateRaster, method='average', align=TRUE)
        # Binarize aggregated cloudmask
        cloudmaskAgg <- ifel(pCloudAgg < 0.2, 1, NA)

        #Only generate if aggregated files do not already exist
        # if(!file_exists(toaOut)) {
        chm <- rast(chmPath)
        # chm100m <- project(chm, template100m_3338, method='average', align=TRUE)
        # chm_agg <- project(chm, templateRaster, method='average', align=TRUE)
        chm_agg <- mask(project(chm,    cloudmaskAgg, method='average', align=TRUE), cloudmaskAgg)
        # chm_dm <- clamp(round(chm_agg * 10), 0, 1200)
        chm_dm <- clamp(round(chm_agg), 0, 1200)
        # project(chm, template100m_3338, method='average', align=TRUE)
        dir_create(path_dir(chmOutTmp), mode="2775")
        writeRaster(chm_dm, filename=chmOutTmp, datatype='INT2U', gdal=c('COMPRESS=DEFLATE','TILED=YES'), NAflag=65535, overwrite=T)
        dir_create(path_dir(chmOut), mode="2775")
        gdal_translate(chmOutTmp, chmOut, of='COG', ot='UInt16', co=c('COMPRESS=DEFLATE','TILED=YES'))
        file_delete(chmOutTmp)
        file_chmod(chmOut, "0775")
        print(chmOut)
        gc()
        }
    }
    #     return(chmOut)
    #     if(!file_exists(chmOut)) {
    #         # Project 4band TOA to match aggregated cloudmask, then apply the mask
    #         # toaAgg <-    mask(project(toa,    cloudmaskAgg, method='average'), cloudmaskAgg)
    #         chmAgg <-    mask(project(chm,    cloudmaskAgg, method='average'), cloudmaskAgg)
    #         # Save result
    #         writeRaster(chmAgg, filename=chmOutTmp, datatype='INT2S', gdal=c('COMPRESS=DEFLATE','TILED=YES'), NAflag=-20000, overwrite=T)
    #         gdal_translate(toaOutTmp, toaOut, of='COG', ot='Int16', co=c('COMPRESS=DEFLATE','TILED=YES'))
    #         file_delete(toaOutTmp)
    #         # writeRaster(toaAgg, filename=toaOutTmp, datatype='INT2S', gdal=c('COMPRESS=DEFLATE','TILED=YES'), NAflag=-20000, overwrite=T)
    #         # gdal_translate(toaOutTmp, toaOut, of='COG', ot='Int16', co=c('COMPRESS=DEFLATE','TILED=YES'))
    #         # file_delete(toaOutTmp)
            
    #         #Now do rastid
    #         # print(toaAgg)
    #         # print(rastid)
    #         # rastid_rast <- anyNA(toaAgg) * rastid
    #         rastid_data <- min(toaAgg) >= -10000
    #         rastid_rast <- (rastid_data > 0) * rastid
    #         # print(rastid_rast)
    #         writeRaster(rastid_rast, filename=rastidOutTmp, datatype='INT2U', gdal=c('COMPRESS=DEFLATE','TILED=YES'), NAflag=0, overwrite=T)
    #         gdal_translate(rastidOutTmp, rastidOut, of='COG', ot='UInt16', co=c('COMPRESS=DEFLATE','TILED=YES'))
    #         file_delete(rastidOutTmp)
    #         }
    #     if(!file_exists(srliteOut)) {
    #         # Project 4band SRLite to match aggregated cloudmask, then apply the mask
    #         srliteAgg <- mask(project(srlite, cloudmaskAgg, method='average'), cloudmaskAgg)
    #         # Save result
    #         writeRaster(srliteAgg, filename=srliteOutTmp, datatype='INT2S', gdal=c('COMPRESS=DEFLATE','TILED=YES'), NAflag=-20000, overwrite=T)
    #         gdal_translate(srliteOutTmp, srliteOut, of='COG', ot='Int16', co=c('COMPRESS=DEFLATE','TILED=YES'))
    #         file_delete(srliteOutTmp)
    #         }
    #     # Garbage cleanup to avoid memory errors
    #     gc()
    #     }
    # return(nameBase)
    # }


In [9]:
# TODO Add/test chmod command to make outputs world readable as they are created
# See block after future_pmap for current permission cleanup syntax
# 2026-01-27 Modified to account for latest CHM inputs being pre-scaled to dm
# chm_toInt16_dm <- function(chmPath, chmOut, chmOutTmp, templateRaster) {
#     # nameBase = str_sub(path_ext_remove(path_file(chmPath)), 0, -10)
#     # chm = path(srliteBase, 'toa/agg100m_cm', glue('{nameBase}-toa-cm-100m.tif'))
#     # srliteOut = path(srliteBase, 'srlite/agg100m_cm', glue('{nameBase}-sr-cm-100m.tif'))
#     if(!file_exists(chmOut)) {
#         chm <- rast(chmPath)
#         # chm100m <- project(chm, template100m_3338, method='average', align=TRUE)
#         chm_agg <- project(chm, templateRaster, method='average', align=TRUE)
#         # chm_dm <- clamp(round(chm_agg * 10), 0, 1200)
#         chm_dm <- clamp(round(chm_agg), 0, 1200)
#         # project(chm, template100m_3338, method='average', align=TRUE)
#         dir_create(path_dir(chmOutTmp), mode="2775")
#         writeRaster(chm_dm, filename=chmOutTmp, datatype='INT2U', gdal=c('COMPRESS=DEFLATE','TILED=YES'), NAflag=65535, overwrite=T)
#         dir_create(path_dir(chmOut), mode="2775")
#         gdal_translate(chmOutTmp, chmOut, of='COG', ot='UInt16', co=c('COMPRESS=DEFLATE','TILED=YES'))
#         file_delete(chmOutTmp)
#         file_chmod(chmOut, "0775")
#         print(chmOut)
#         gc()
#         }
#     return(chmOut)
#     }


In [10]:
# agg_and_cloudmask <- function(chmPath, cmPath, chmOut, chmOutTmp, templateRaster) { #, resName) {

chm_dtm_params <- unique_joined %>% select(chmPath, cmPath, chmOut = chmDmOut, chmOutTmp = chmDmOutTmp) #, template010m_3338)
head(chm_dtm_params)
datatable(chm_dtm_params)

chmPath,cmPath,chmOut,chmOutTmp
<fs::path>,<fs::path>,<fs::path>,<fs::path>
/explore/nobackup/projects/above/misc/ABoVE_Shrubs/development/chm/dinov3/4.3.2.5/WV02_20100502_M1BS_10300100045FA300-sr-02m.chm.tif,/explore/nobackup/projects/above/misc/ABoVE_Shrubs/cloudmask/omni_cloudmask/002m_2023/WV02_20100502_M1BS_10300100045FA300-sr-02m_OCM_v1_6_0.tif,/explore/nobackup/projects/above/misc/ABoVE_Shrubs/chm/2026_chm/4.3.2.5/dm_10m/WV02_20100502_M1BS_10300100045FA300-chm-dm-10m.tif,/explore/nobackup/projects/above/misc/ABoVE_Shrubs/chm/2026_chm/4.3.2.5/dm_10m/WV02_20100502_M1BS_10300100045FA300-chm-dm-10m-tmp.tif
/explore/nobackup/projects/above/misc/ABoVE_Shrubs/development/chm/dinov3/4.3.2.5/WV02_20100505_M1BS_1030010005215900-sr-02m.chm.tif,/explore/nobackup/projects/above/misc/ABoVE_Shrubs/cloudmask/omni_cloudmask/002m_2023/WV02_20100505_M1BS_1030010005215900-sr-02m_OCM_v1_6_0.tif,/explore/nobackup/projects/above/misc/ABoVE_Shrubs/chm/2026_chm/4.3.2.5/dm_10m/WV02_20100505_M1BS_1030010005215900-chm-dm-10m.tif,/explore/nobackup/projects/above/misc/ABoVE_Shrubs/chm/2026_chm/4.3.2.5/dm_10m/WV02_20100505_M1BS_1030010005215900-chm-dm-10m-tmp.tif
/explore/nobackup/projects/above/misc/ABoVE_Shrubs/development/chm/dinov3/4.3.2.5/WV02_20100525_M1BS_1030010005429600-sr-02m.chm.tif,/explore/nobackup/projects/above/misc/ABoVE_Shrubs/cloudmask/omni_cloudmask/002m_2023/WV02_20100525_M1BS_1030010005429600-sr-02m_OCM_v1_6_0.tif,/explore/nobackup/projects/above/misc/ABoVE_Shrubs/chm/2026_chm/4.3.2.5/dm_10m/WV02_20100525_M1BS_1030010005429600-chm-dm-10m.tif,/explore/nobackup/projects/above/misc/ABoVE_Shrubs/chm/2026_chm/4.3.2.5/dm_10m/WV02_20100525_M1BS_1030010005429600-chm-dm-10m-tmp.tif
/explore/nobackup/projects/above/misc/ABoVE_Shrubs/development/chm/dinov3/4.3.2.5/WV02_20100526_M1BS_1030010005890800-sr-02m.chm.tif,/explore/nobackup/projects/above/misc/ABoVE_Shrubs/cloudmask/omni_cloudmask/002m_2023/WV02_20100526_M1BS_1030010005890800-sr-02m_OCM_v1_6_0.tif,/explore/nobackup/projects/above/misc/ABoVE_Shrubs/chm/2026_chm/4.3.2.5/dm_10m/WV02_20100526_M1BS_1030010005890800-chm-dm-10m.tif,/explore/nobackup/projects/above/misc/ABoVE_Shrubs/chm/2026_chm/4.3.2.5/dm_10m/WV02_20100526_M1BS_1030010005890800-chm-dm-10m-tmp.tif
/explore/nobackup/projects/above/misc/ABoVE_Shrubs/development/chm/dinov3/4.3.2.5/WV02_20100529_M1BS_10300100054FCA00-sr-02m.chm.tif,/explore/nobackup/projects/above/misc/ABoVE_Shrubs/cloudmask/omni_cloudmask/002m_2023/WV02_20100529_M1BS_10300100054FCA00-sr-02m_OCM_v1_6_0.tif,/explore/nobackup/projects/above/misc/ABoVE_Shrubs/chm/2026_chm/4.3.2.5/dm_10m/WV02_20100529_M1BS_10300100054FCA00-chm-dm-10m.tif,/explore/nobackup/projects/above/misc/ABoVE_Shrubs/chm/2026_chm/4.3.2.5/dm_10m/WV02_20100529_M1BS_10300100054FCA00-chm-dm-10m-tmp.tif
/explore/nobackup/projects/above/misc/ABoVE_Shrubs/development/chm/dinov3/4.3.2.5/WV02_20100531_M1BS_10300100053C1900-sr-02m.chm.tif,/explore/nobackup/projects/above/misc/ABoVE_Shrubs/cloudmask/omni_cloudmask/002m_2023/WV02_20100531_M1BS_10300100053C1900-sr-02m_OCM_v1_6_0.tif,/explore/nobackup/projects/above/misc/ABoVE_Shrubs/chm/2026_chm/4.3.2.5/dm_10m/WV02_20100531_M1BS_10300100053C1900-chm-dm-10m.tif,/explore/nobackup/projects/above/misc/ABoVE_Shrubs/chm/2026_chm/4.3.2.5/dm_10m/WV02_20100531_M1BS_10300100053C1900-chm-dm-10m-tmp.tif


Warning message in instance$preRenderHook(instance):
“It seems your data is too big for client-side DataTables. You may consider server-side processing: https://rstudio.github.io/DT/server.html”


HTML widgets cannot be represented in plain text (need html)

In [11]:
# chm_toInt16_dm('/explore/nobackup/people/mmacande/srlite/chm_model/20231014_chm/products_merged_nodtm_train_strips/WV02_20100804_M1BS_103001000617C000-sr-02m/WV02_20100804_M1BS_103001000617C000-sr-02m.cnn-chm-v1.tif', 
#         '/explore/nobackup/projects/above/misc/ABoVE_Shrubs/chm/20231014_chm/dm_10m/WV02_20100804_M1BS_103001000617C000-chm_dm_010m.tif',
#         template010m_3338)

In [12]:
#Don't think random number generation within parallel is an issue here so quiet the warnings. See below
#https://www.r-bloggers.com/2020/09/future-1-19-1-making-sure-proper-random-numbers-are-produced-in-parallel-processing/
options(future.rng.onMisuse = "ignore")

In [13]:
plan(multicore, workers=16)
# plan(multicore, workers=16)

In [14]:
plan()

multicore:
- args: function (..., workers = 16, envir = parent.frame())
- tweaked: TRUE
- call: plan(multicore, workers = 16)

### Run batch

In [15]:
# agg_chm_list <- future_pmap(chm_dtm_params %>% slice_sample(n=10),
agg_chm_list <- future_pmap(chm_dtm_params, 
                                        possibly(agg_and_cloudmask, otherwise=NULL),
                                        templateRaster = template010m_3338)

/explore/nobackup/projects/above/misc/ABoVE_Shrubs/chm/2026_chm/4.3.2.5/dm_10m/WV02_20130618_M1BS_1030010023C47A00-chm-dm-10m.tif
/explore/nobackup/projects/above/misc/ABoVE_Shrubs/chm/2026_chm/4.3.2.5/dm_10m/WV02_20130619_M1BS_1030010023185100-chm-dm-10m.tif
/explore/nobackup/projects/above/misc/ABoVE_Shrubs/chm/2026_chm/4.3.2.5/dm_10m/WV02_20130619_M1BS_1030010023456800-chm-dm-10m.tif
/explore/nobackup/projects/above/misc/ABoVE_Shrubs/chm/2026_chm/4.3.2.5/dm_10m/WV02_20130730_M1BS_1030010026ADA200-chm-dm-10m.tif
/explore/nobackup/projects/above/misc/ABoVE_Shrubs/chm/2026_chm/4.3.2.5/dm_10m/WV02_20130731_M1BS_1030010025861E00-chm-dm-10m.tif
/explore/nobackup/projects/above/misc/ABoVE_Shrubs/chm/2026_chm/4.3.2.5/dm_10m/WV02_20130731_M1BS_10300100258D8600-chm-dm-10m.tif
/explore/nobackup/projects/above/misc/ABoVE_Shrubs/chm/2026_chm/4.3.2.5/dm_10m/WV02_20130731_M1BS_1030010025B6C800-chm-dm-10m.tif
/explore/nobackup/projects/above/misc/ABoVE_Shrubs/chm/2026_chm/4.3.2.5/dm_10m/WV02_201307

In [ ]:
# 1. Permission fix, now done in function
# target_dirs <- unique(path(srliteBase, 'chm/2026_chm/4.3.2.5/dm_10m', chms_predicted_2026_fs$chmSourceShort))

# target_dirs
# file_chmod(target_dirs, "0775")

# dir_ls(target_dirs, recurse = TRUE) %>%
#   file_chmod("0775")

In [ ]:
datatable(head(agg_chm_list))
print(c('Number of aggregated/cloudmasked images:', nrow(agg_chm_list)))
print(c('Number of chm outputs:', length(dir_ls('/explore/nobackup/projects/above/misc/ABoVE_Shrubs/chm/2026_chm/4.3.2.5/dm_10m', glob='*.tif'))))
# print(c('Number of srlite outputs:', length(dir_ls('/explore/nobackup/projects/above/misc/ABoVE_Shrubs/srlite/agg010m_cm', glob='*.tif'))))
# print(c('Number of rastid outputs:', length(dir_ls('/explore/nobackup/projects/above/misc/ABoVE_Shrubs/rastid/agg010m_cm', glob='*.tif'))))


In [ ]:
# toa_agg <- tibble(toaPath = dir_ls('/explore/nobackup/projects/above/misc/ABoVE_Shrubs/toa/agg010m_cm', glob='*.tif')) %>%
# mutate(toaFile = path_file(toaPath),
#        catid = str_sub(toaFile, 20, 35))
# # dir_ls('/explore/nobackup/projects/above/misc/ABoVE_Shrubs/srlite/agg010m_cm', glob='*.tif')
# # dir_ls('/explore/nobackup/projects/above/misc/ABoVE_Shrubs/rastid/agg010m_cm', glob='*.tif')
# head(toa_agg)
